# Seizure Predicition Procedure
1.   Combine all EEG signal channels using Averaging
2.   Filter using bass band filters for increasing SNR
3.   Feature extraction
4.   Normalization
5.   Balancing the dataset
6.   SVM Vs. KNN for classification
7. Testing & validation using k-fold



# Importing and installing needed libraries & setting google drive folder


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import abs, sum, linspace
from numpy.fft import rfft
import librosa, librosa.display

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#Ready surrogate channel( CSV files containing averaged output)

In [ ]:
patient1_3Surrogate_df= pd.read_csv('/content/drive/My Drive/Graduationnn/trial/patient1_3Surrogate_df.csv')
patient1_3Surrogate_np= patient1_3Surrogate_df['0'].to_numpy()

patient1_4Surrogate_df= pd.read_csv('/content/drive/My Drive/Graduationnn/trial/patient1_4Surrogate_df.csv')
patient1_4Surrogate_np= patient1_4Surrogate_df['0'].to_numpy()

patient1_15Surrogate_df= pd.read_csv('/content/drive/My Drive/Graduationnn/trial/patient1_15Surrogate_df.csv')
patient1_15Surrogate_np= patient1_15Surrogate_df['0'].to_numpy()

patient1_16Surrogate_df= pd.read_csv('/content/drive/My Drive/Graduationnn/trial/patient1_16Surrogate_df.csv')
patient1_16Surrogate_np= patient1_16Surrogate_df['0'].to_numpy()

patient1_18Surrogate_df= pd.read_csv('/content/drive/My Drive/Graduationnn/trial/patient1_18Surrogate_df.csv')
patient1_18Surrogate_np= patient1_18Surrogate_df['0'].to_numpy()

patient1_21Surrogate_df= pd.read_csv('/content/drive/My Drive/Graduationnn/trial/patient1_21Surrogate_df.csv')
patient1_21Surrogate_np= patient1_21Surrogate_df['0'].to_numpy()

patient3_2Surrogate_df= pd.read_csv('/content/drive/My Drive/Graduationnn/trial/patient3_2Surrogate_df.csv')
patient3_2Surrogate_np= patient3_2Surrogate_df['0'].to_numpy()

patient5_6Surrogate_df= pd.read_csv('/content/drive/My Drive/Graduationnn/trial/patient5_6Surrogate_df.csv')
patient5_6Surrogate_np= patient5_6Surrogate_df['0'].to_numpy()

patient5_13Surrogate_df= pd.read_csv('/content/drive/My Drive/Graduationnn/trial/patient5_13Surrogate_df.csv')
patient5_13Surrogate_np= patient5_13Surrogate_df['0'].to_numpy()

patient5_16Surrogate_df= pd.read_csv('/content/drive/My Drive/Graduationnn/trial/patient5_16Surrogate_df.csv')
patient5_16Surrogate_np= patient5_16Surrogate_df['0'].to_numpy()

patient12_09Surrogate_df= pd.read_csv('/content/drive/My Drive/Graduationnn/trial/patient12_09Surrogate_df.csv')
patient12_09Surrogate_np= patient12_09Surrogate_df['0'].to_numpy()

patient18_29Surrogate_df= pd.read_csv('/content/drive/My Drive/Graduationnn/trial/patient18_29Surrogate_df.csv')
patient18_29Surrogate_np= patient18_29Surrogate_df['0'].to_numpy()

patient18_31Surrogate_df= pd.read_csv('/content/drive/My Drive/Graduationnn/trial/patient18_31Surrogate_df.csv')
patient18_31Surrogate_np= patient18_31Surrogate_df['0'].to_numpy()

patient24_04Surrogate_df= pd.read_csv('/content/drive/My Drive/Graduationnn/trial/patient24_04Surrogate_df.csv')
patient24_04Surrogate_np= patient24_04Surrogate_df['0'][210:].to_numpy()


#Filtering

In [ ]:
from scipy.signal import butter, lfilter

def butter_bandpass(lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [ ]:
def signalFilter(data, fs):
  delta = butter_bandpass_filter(data, 0.5, 4, fs, order=4)
  theta = butter_bandpass_filter(data, 4, 8, fs, order=4)
  alpha = butter_bandpass_filter(data, 8, 12, fs, order=4)
  beta = butter_bandpass_filter(data, 12, 25, fs, order=4)
  return delta,theta,alpha,beta

#Featurs Extraction

In [ ]:
#Statistical
def statisticalMean(signal):
  return abs(np.mean(signal))

def statisticalSTD(signal):
  return np.std(signal)

def statisticalSkewness(signal):
  dataframe = pd.DataFrame(signal)
  return dataframe.skew()[0]

def statisticalKurtosis(signal):
  dataframe = pd.DataFrame(signal)
  return dataframe.kurtosis()[0]

In [ ]:
#Spectral
def spectralCentroid(signal):
  #It is calculated as the weighted mean of the frequencies present in the signal, 
  #determined using a Fourier transform, with their magnitudes as the weights.
  n = len(signal)
  timestep = 1/256
  spectrum = abs(np.fft.rfft(signal)) #using FT to compute frequcies present in the signal
  normalized_spectrum = spectrum / sum(spectrum)  # like a probability mass function
  normalized_frequencies = np.linspace(0, 1, len(spectrum))
  spectral_centroid = sum(normalized_frequencies * normalized_spectrum)
  freq = np.fft.rfftfreq(n, d=timestep) #get frequncies
  freq_max= freq.max() 
  # Spectral centroid is calculated as a ratio.  Multiply by your maximum frequency bin to get real frequency.
  spectral_centroid= freq_max * spectral_centroid
  return spectral_centroid

def averageEnergy(signal):
  summation = 0
  for i in range(len(signal)):
    summation = summation + ((signal[i])**2)
  avg_energy = summation/ (len(signal))
  return avg_energy

In [ ]:
def divide_and_filter(signal,frame_length,hop_length):
  
  signal= butter_bandpass_filter(signal, lowcut=0.5, highcut=25, fs=256, order=4)
  frames = librosa.util.frame(signal, frame_length=frame_length, hop_length=hop_length)
  
  e_dataframe= pd.DataFrame()
  std_l=[]
  skew_l=[]
  kurtosis_l=[]
  spC_l=[]
  deltaList= []
  thetaList= []
  alphaList= []
  betaList= []
  
  for frame in frames.T:
    delta,theta,alpha,beta = signalFilter(frame, 256)
    deltaList.append(averageEnergy(delta))
    thetaList.append(averageEnergy(theta))
    alphaList.append(averageEnergy(alpha))
    betaList.append(averageEnergy(beta))
    
    stat_std= statisticalSTD(frame)
    std_l.append(stat_std)

    stat_skew= statisticalSkewness(frame)
    skew_l.append(stat_skew)

    stat_kurtosis= statisticalKurtosis(frame)
    kurtosis_l.append(stat_kurtosis)
    
    spec_cetroid= spectralCentroid(frame)
    spC_l.append(spec_cetroid)

  e_dataframe['Standard Deviation']= std_l
  e_dataframe['Skewness']= skew_l
  e_dataframe['Kurtosis']= kurtosis_l
  e_dataframe['Spectral Centroid']= spC_l
  e_dataframe['delta'] = deltaList
  e_dataframe['theta'] = thetaList
  e_dataframe['alpha'] = alphaList
  e_dataframe['beta'] = betaList

  return e_dataframe

#Normalization (feature Scaling)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
def normalizeData(dataFrame):
  minRange = 0
  maxRange = 100
  scaler = MinMaxScaler(feature_range=(minRange, maxRange), copy=True)
  df = pd.DataFrame(scaler.fit_transform(dataFrame),columns=dataFrame.columns, index=dataFrame.index) 
  return df

#Testing

In [ ]:
#patient1_3h
patient1_3h_ex= divide_and_filter(patient1_3Surrogate_np,2560,2560)
patient1_3h_ex_norm= normalizeData(patient1_3h_ex)

#patient1_4h
patient1_4h_ex= divide_and_filter(patient1_4Surrogate_np,2560,2560)
patient1_4h_ex_norm= normalizeData(patient1_4h_ex)

#patient1_15h
patient1_15h_ex= divide_and_filter(patient1_15Surrogate_np,2560,2560)
patient1_15h_ex_norm= normalizeData(patient1_15h_ex)

#patient1_16h
patient1_16h_ex= divide_and_filter(patient1_16Surrogate_np,2560,2560)
patient1_16h_ex_norm= normalizeData(patient1_16h_ex)

#patient1_18h
patient1_18h_ex= divide_and_filter(patient1_18Surrogate_np,2560,2560)
patient1_18h_ex_norm= normalizeData(patient1_18h_ex)

#patient1_21h
patient1_21h_ex= divide_and_filter(patient1_21Surrogate_np,2560,2560)
patient1_21h_ex_norm= normalizeData(patient1_21h_ex)

#patient3_2h
patient3_2h_ex= divide_and_filter(patient3_2Surrogate_np,2560,2560)
patient3_2h_ex_norm= normalizeData(patient3_2h_ex)
#patient 12_9h
patient12_9h_ex= divide_and_filter(patient12_09Surrogate_np,2560,2560)
patient12_9h_ex_norm= normalizeData(patient12_9h_ex)

#patient5
patient5_6h_ex= divide_and_filter(patient5_6Surrogate_np,2560,2560)
patient5_6h_ex_norm= normalizeData(patient5_6h_ex)

patient5_13h_ex= divide_and_filter(patient5_13Surrogate_np,2560,2560)
patient5_13h_ex_norm= normalizeData(patient5_13h_ex)

patient5_16h_ex= divide_and_filter(patient5_16Surrogate_np,2560,2560)
patient5_16h_ex_norm= normalizeData(patient5_16h_ex)

#patient 18_29h
patient18_29h_ex= divide_and_filter(patient18_29Surrogate_np,2560,2560)
patient18_29h_ex_norm= normalizeData(patient18_29h_ex)

#patient 18_31h
patient18_31h_ex= divide_and_filter(patient18_31Surrogate_np,2560,2560)
patient18_31h_ex_norm= normalizeData(patient18_31h_ex)

#patient 24_4h
patient24_4h_ex= divide_and_filter(patient24_04Surrogate_np,2560,2560)
patient24_4h_ex_norm= normalizeData(patient24_4h_ex)

In [ ]:
patient1_3h_ex_norm["Label"] = 0
patient1_3h_ex_norm.iloc[300:303]['Label']=1

patient1_4h_ex_norm["Label"] = 0
patient1_4h_ex_norm.iloc[146:151]['Label']=1

patient1_15h_ex_norm["Label"] = 0
patient1_15h_ex_norm.iloc[173:177]['Label']=1

patient1_16h_ex_norm["Label"] = 0
patient1_16h_ex_norm.iloc[100:106]['Label']=1

patient1_18h_ex_norm["Label"] = 0
patient1_18h_ex_norm.iloc[172:181]['Label']=1

patient1_21h_ex_norm["Label"] = 0
patient1_21h_ex_norm.iloc[33:42]['Label']=1

patient3_2h_ex_norm["Label"] = 0
patient3_2h_ex_norm.iloc[73:80]['Label']=1

patient12_9h_ex_norm["Label"] = 0
patient12_9h_ex_norm.iloc[308:311]['Label']=1

patient5_6h_ex_norm["Label"] = 0
patient5_6h_ex_norm.iloc[46:54]['Label']=1

patient5_13h_ex_norm["Label"] = 0
patient5_13h_ex_norm.iloc[111:120]['Label']=1

patient5_16h_ex_norm["Label"] = 0
patient5_16h_ex_norm.iloc[235:241]['Label']=1

patient18_29h_ex_norm["Label"] = 0
patient18_29h_ex_norm.iloc[349:353]['Label']=1

patient18_31h_ex_norm["Label"] = 0
patient18_31h_ex_norm.iloc[208:213]['Label']=1

patient24_4h_ex_norm["Label"] = 0

In [ ]:
frames= [patient1_3h_ex_norm[280:320], patient1_4h_ex_norm[130:170],patient1_15h_ex_norm[150:190],patient1_16h_ex_norm[80:120],patient1_18h_ex_norm[155:195],
         patient1_21h_ex_norm[15:55], patient3_2h_ex_norm[55:95],patient12_9h_ex_norm[280:320], patient5_6h_ex_norm[25:65], patient5_13h_ex_norm[95:135], 
         patient5_16h_ex_norm[215:255],patient18_29h_ex_norm[313:353], patient18_31h_ex_norm[176:216], patient24_4h_ex_norm[0:20]]
result = pd.concat(frames)
result=result.reset_index()
result=result.drop(columns='index')
result.shape

(540, 9)

In [ ]:
result.head(5)

,Standard Deviation,Skewness,Kurtosis,Spectral Centroid,delta,theta,alpha,beta,Label
0,5.984302,51.877290,19.518346,34.460581,0.919281,4.593203,18.575051,11.474303,0
1,9.379791,46.531777,8.659403,25.335442,1.943445,4.248461,28.115566,39.160667,0
2,13.030767,50.193374,38.259478,32.286168,3.619233,4.276488,21.241020,25.410534,0
3,3.729856,41.554274,8.201615,26.018449,0.509283,2.907127,16.505304,16.966855,0
4,13.971547,53.989292,26.815773,10.311566,4.158961,2.801256,16.613062,31.270486,0


# Support Vector Machine Vs. KNN Classification

#All patients

In [ ]:
#Trial 1: Band bass filter & time and spectral old features
#SVM
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold 
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score


svclassifier = SVC(kernel='linear',C=1)

X = result.drop(columns=['Label'],axis=1)
y = result['Label']
# KFold Cross Validation approach
kf = KFold(n_splits=5,shuffle=False)
kf.split(X)    
     
# Initialize the accuracy of the models to blank list. The accuracy of each model will be appended to this list
accuracy_model = []
 
# Iterate over each train-test split
cm_holder=[]
total=np.empty([2, 2])

for train_index, test_index in kf.split(X):
    # Split train-test
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Train the model
    model = svclassifier.fit(X_train, y_train)
    y_predict = svclassifier.predict(X_test)
    print("new Matrix")
    print(confusion_matrix(y_test,y_predict))
    # Append to accuracy_model the accuracy of the model
    cm_holder.append(confusion_matrix(y_test, y_predict))
    accuracy_model.append(accuracy_score(y_test, model.predict(X_test), normalize=True)*100)
for i in range(len(cm_holder)):
  total= total+ cm_holder[i]
print("the total confusion matrix is", total)   
#print(accuracy_model)

new Matrix
[[589   0]
 [  3   5]]
new Matrix
[[568   0]
 [ 13  15]]
new Matrix
[[570   2]
 [  2  22]]
new Matrix
[[584   3]
 [  3   6]]
new Matrix
[[576  11]
 [  3   6]]
the total confusion matrix is [[2887.   16.]
 [  24.   54.]]


In [ ]:
results of svm 
specificity= 0.99
sensitivity= 0.70
accuracy= 0.98

In [ ]:
#Trial 4: Band bass filter & all features without mean and average energy.
#SVM
from sklearn.svm import SVC
from sklearn.model_selection import KFold 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score


svclassifier = SVC(kernel='linear',C=1)

X = result.drop(columns=['Label'],axis=1)
y = result['Label']

# KFold Cross Validation approach
kf = KFold(n_splits=5,shuffle=False)
kf.split(X)    
     
# Initialize the accuracy of the models to blank list. The accuracy of each model will be appended to this list
accuracy_model = []
 
# Iterate over each train-test split
cm_holder=[]
total=np.empty([2, 2])

for train_index, test_index in kf.split(X):
    # Split train-test
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Train the model
    model = svclassifier.fit(X_train, y_train)
    y_predict = svclassifier.predict(X_test)
    print("new Matrix")
    print(confusion_matrix(y_test,y_predict))
    # Append to accuracy_model the accuracy of the model
    cm_holder.append(confusion_matrix(y_test, y_predict))
    accuracy_model.append(accuracy_score(y_test, model.predict(X_test), normalize=True)*100)
for i in range(len(cm_holder)):
  total= total+ cm_holder[i]
print("the total confusion matrix is", total)   

new Matrix
[[589   0]
 [  2   6]]
new Matrix
[[568   0]
 [  6  22]]
new Matrix
[[570   2]
 [  8  16]]
new Matrix
[[579   8]
 [  2   7]]
new Matrix
[[581   6]
 [  1   8]]
the total confusion matrix is [[2887.   16.]
 [  19.   59.]]


In [ ]:
results of svm 
specificity= 0.99
sensitivity= 0.76
accuracy= 0.988

In [ ]:
#Trial 4: Band bass filter & all features without mean and average energy.
#SVM
# resampling: downsizing >> ratio: 0.14
from sklearn.svm import SVC
from sklearn.model_selection import KFold 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score


svclassifier = SVC(kernel='linear',C=1)

X = result.drop(columns=['Label'],axis=1)
y = result['Label']

# KFold Cross Validation approach
kf = KFold(n_splits=5,shuffle=False)
kf.split(X)    
     
# Initialize the accuracy of the models to blank list. The accuracy of each model will be appended to this list
accuracy_model = []
 
# Iterate over each train-test split
cm_holder=[]
total=np.empty([2, 2])

for train_index, test_index in kf.split(X):
    # Split train-test
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Train the model
    model = svclassifier.fit(X_train, y_train)
    y_predict = svclassifier.predict(X_test)
    print("new Matrix")
    print(confusion_matrix(y_test,y_predict))
    # Append to accuracy_model the accuracy of the model
    cm_holder.append(confusion_matrix(y_test, y_predict))
    accuracy_model.append(accuracy_score(y_test, model.predict(X_test), normalize=True)*100)
for i in range(len(cm_holder)):
  total= total+ cm_holder[i]
print("the total confusion matrix is", total)   

new Matrix
[[95  1]
 [ 2 10]]
new Matrix
[[91  2]
 [ 2 13]]
new Matrix
[[84  5]
 [ 2 17]]
new Matrix
[[83  2]
 [ 7 16]]
new Matrix
[[98  1]
 [ 1  8]]
the total confusion matrix is [[451.  11.]
 [ 14.  64.]]


In [ ]:
#Trial 4: Band bass filter & all features without mean and average energy.
#knn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold 
from sklearn.metrics import accuracy_score

classifier = KNeighborsClassifier(n_neighbors=5)
X = result.drop(columns=['Label'],axis=1)
y = result['Label']

# KFold Cross Validation approach
kf = KFold(n_splits=5,shuffle=False)
kf.split(X)    
     
# Initialize the accuracy of the models to blank list. The accuracy of each model will be appended to this list
accuracy_model = []
 
# Iterate over each train-test split
cm_holder=[]
total=np.empty([2, 2])

for train_index, test_index in kf.split(X):
    # Split train-test
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Train the model
    model = classifier.fit(X_train, y_train)
    y_predict = classifier.predict(X_test)
    print("new Matrix")
    print(confusion_matrix(y_test,y_predict))
    # Append to accuracy_model the accuracy of the model
    cm_holder.append(confusion_matrix(y_test, y_predict))
    accuracy_model.append(accuracy_score(y_test, model.predict(X_test), normalize=True)*100)
for i in range(len(cm_holder)):
  total= total+ cm_holder[i]
print("the total confusion matrix is", total)   
print(accuracy_model)

new Matrix
[[588   1]
 [  3   5]]
new Matrix
[[568   0]
 [  6  22]]
new Matrix
[[570   2]
 [ 10  14]]
new Matrix
[[582   5]
 [  2   7]]
new Matrix
[[581   6]
 [  1   8]]
the total confusion matrix is [[2889.   14.]
 [  22.   56.]]
[99.32998324958125, 98.99328859060402, 97.98657718120806, 98.8255033557047, 98.8255033557047]


In [ ]:
results of KNN 
specificity= 0.99
sensitivity= 0.72
accuracy= 0.987

#Single Patient

In [ ]:
frames_1= [patient1_3h_ex_norm[280:320], patient1_4h_ex_norm[130:170],patient1_15h_ex_norm[150:190],patient1_16h_ex_norm[80:120],patient1_18h_ex_norm[155:195],
         patient1_21h_ex_norm[15:55]]
result = pd.concat(frames_1)
result=result.reset_index()
result=result.drop(columns='index')
result.shape

(240, 9)

In [ ]:
#Trial 4: Band bass filter & all features without mean and average energy.
#SVM
from sklearn.svm import SVC
from sklearn.model_selection import KFold 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score


svclassifier = SVC(kernel='linear',C=1)

X = result.drop(columns=['Label'],axis=1)
y = result['Label']

# KFold Cross Validation approach
kf = KFold(n_splits=3,shuffle=False)
kf.split(X)    
     
# Initialize the accuracy of the models to blank list. The accuracy of each model will be appended to this list
accuracy_model = []
 
# Iterate over each train-test split
cm_holder=[]
total=np.empty([2, 2])

for train_index, test_index in kf.split(X):
    # Split train-test
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Train the model
    model = svclassifier.fit(X_train, y_train)
    y_predict = svclassifier.predict(X_test)
    print("new Matrix")
    print(confusion_matrix(y_test,y_predict))
    # Append to accuracy_model the accuracy of the model
    cm_holder.append(confusion_matrix(y_test, y_predict))
    accuracy_model.append(accuracy_score(y_test, model.predict(X_test), normalize=True)*100)
for i in range(len(cm_holder)):
  total= total+ cm_holder[i]
print("the total confusion matrix is", total)   

new Matrix
[[417   0]
 [  0   3]]
new Matrix
[[401   4]
 [  2  13]]
new Matrix
[[402   0]
 [  3  15]]
the total confusion matrix is [[1220.    4.]
 [   5.   31.]]


In [ ]:
results of SVM for 1 patient
specificity= 0.98
sensitivity= 0.86
accuracy= 0.98

In [ ]:
#Trial 4: Band bass filter & all features without mean and average energy.
#SVM
#down sampling
from sklearn.svm import SVC
from sklearn.model_selection import KFold 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score


svclassifier = SVC(kernel='linear',C=1)

X = result.drop(columns=['Label'],axis=1)
y = result['Label']

# KFold Cross Validation approach
kf = KFold(n_splits=3,shuffle=False)
kf.split(X)    
     
# Initialize the accuracy of the models to blank list. The accuracy of each model will be appended to this list
accuracy_model = []
 
# Iterate over each train-test split
cm_holder=[]
total=np.empty([2, 2])

for train_index, test_index in kf.split(X):
    # Split train-test
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Train the model
    model = svclassifier.fit(X_train, y_train)
    y_predict = svclassifier.predict(X_test)
    print("new Matrix")
    print(confusion_matrix(y_test,y_predict))
    # Append to accuracy_model the accuracy of the model
    cm_holder.append(confusion_matrix(y_test, y_predict))
    accuracy_model.append(accuracy_score(y_test, model.predict(X_test), normalize=True)*100)
for i in range(len(cm_holder)):
  total= total+ cm_holder[i]
print("the total confusion matrix is", total)   

new Matrix
[[72  0]
 [ 2  6]]
new Matrix
[[64  6]
 [ 0 10]]
new Matrix
[[62  0]
 [ 1 17]]
the total confusion matrix is [[198.   6.]
 [  3.  33.]]


In [ ]:
results of SVM for 1 patient
specificity= 0.98
sensitivity= 0.91
accuracy= 0.98